In [1]:
#HunderPlus Recommender System

#This algorithm is used to predict the recommended procucts based on the current product displayed on the product display page.
#No external libraries are used in this except the mysql.connector to get the connection to the sql database,
#radom class and math class.

#Note that in this algorithm I've combined both the databases i.e., the one containing the ORDERS and the other
#containing the PRODUCTS/CATEGORIES schemas.
#While writing the php code be sure to reference the correct database when needed.

#You can use the Jupyter notebook to run this or even Google colab/ Kaggle notebooks.

In [2]:
#Before continuing with the algorithm, you must know that any query result returned by the mysql connector returns the results
#in a tuple rather than a list, as the tuples in python are immutable we will first extract the contents of the tuple into
#a python list and then perform the desired computation.

In [3]:
#All the print statements except the final list of recommended products are commented out to produce the result faster
#You can remove the '#' from print statements to see the intermediate results.

In [4]:
#Importing libraries
import mysql.connector
import numpy as np
import math
import random

In [5]:
#Utility functions used

#This function is used to remove duplicate entries from a python list
def remove_duplicates(input_list):
    final_list = [] 
    for num in input_list: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list


In [6]:
def recommend_products(id_given):


    #Supposing the product description page is of the product id = 669 (while writing php script this should have the value
    #of the current id of the product on the displayed page.
    
    #print("Product displayed: ")
    cursor.execute("select product_id, name from shop_product_description where product_id ="+id_given+";")
    displayed_product = cursor.fetchall()

    #As you can see this print statement would print the result that is in a tuple
    #print(displayed_product)
    displayed_product_name = displayed_product[0]     #0th index contains the product id
    displayed_product_name = displayed_product_name[1]     #1st index contains the product name

    #now we will fetch the name of the brand of the product i.e., the first word of the string
    displayed_product_brand = displayed_product_name.split(',')
    displayed_product_brand = displayed_product_name.split(' ')
    displayed_product_brand = displayed_product_brand[0]
    #print("BRAND : "+displayed_product_brand)
    
    
    #Step1: Fetching products that shares the same keywords as the displayed product
    
    #Keywords related to the product
    related_keywords = []

    #print("Related keywords: ")
    cursor.execute("select keyword from shop_product_description where product_id ="+id_given+";")
    keywords = cursor.fetchall()

    #when there are no keywords in the keyword field of the product in the database
    if keywords is None:
        keyword_related_products = []

    else:
        for keyword in keywords:
            related_keywords.append(keyword[0])

        #The keywords are splitted by , and || in the schema.
        related_keywords = related_keywords[0].split(',')
        related_keywords = related_keywords[0].split('||')

        for i in range(len(related_keywords)):
            related_keywords[i] = related_keywords[i].strip()

        #print(related_keywords
        #It is seen that only the first 3 keywords in the field are needed.
        related_keywords = related_keywords[0:3]
        #print("\nFiltered related keywords: ")
        #print(related_keywords)
        #print("\nKeyword related prouducts: ")
        keyword_related_products = []
        for keyword in related_keywords:
            query = '''select product_id from shop_product_description where keyword like "%'''+keyword+'''%"'''
            cursor.execute(query)
            keyword_related_products.append(cursor.fetchall())    
        keyword_related_products = keyword_related_products[0]
        keyword_related_products = [item for t in keyword_related_products for item in t]
        #print(keyword_related_products)
    
    
    #Step2: Retrieving the categories that the product is related to
    
    query = "select category_id from shop_product where id="+id_given+";"
    cursor.execute(query)
    categories = cursor.fetchall()

    #Converting the returned query into a python list to work upon
    tuple_elements = []
    for tuple in categories:
        tuple_elements.append(tuple[0])

    #print(tuple_elements)
    categories_list = tuple_elements[0].split(',')
    #print("\nRelated category id's: ")
    #print(categories_list)
    
    #Fetching the category names according to the category ID's
    #The '+' can be used to concatinate two or more lists in python
    category_names = []
    for category in categories_list:
        query = "select name from shop_category_description where shop_category_id ="+category+";"
        cursor.execute(query)
        category_names = category_names + cursor.fetchall()

    #Retrieving values from tuples to a python list to work upom
    category_names = [item for t in category_names for item in t]
    #print(category_names)
    
    
    
    #STEP3: Retrieving category related products
    
    category_related_products = []
    for product in categories_list:
            query = """select id from shop_product where category_id like '%"""+product+"""%';"""
            cursor.execute(query)
            category_related_products.append(cursor.fetchall())

    #Retrieving the values from resulting tuples and storing them into python list
    category_related_products = category_related_products[0]
    category_related_products = [item for t in category_related_products for item in t]
    #print("\nCategory related products: ")
    #print(category_related_products)
    
    
    
    #STEP4: Retrieving most sold products 
    
    query = """select product_id from shop_order_detail group by(product_id) order by count(product_id) DESC;"""
    cursor.execute(query)
    most_sold_products = cursor.fetchall()
    most_sold_products = most_sold_products[0:40]
    most_sold_products = [item for t in most_sold_products for item in t]
    #print("\nMost sold products: ")
    #print(most_sold_products)
    
    
    #STEP5: Retrieving most viewed products
    
    query = """select id from shop_product order by shop_product.view DESC;"""
    cursor.execute(query)
    most_viewed_products = cursor.fetchall()
    most_viewed_products = most_viewed_products[0:40]
    most_viewed_products = [item for t in most_viewed_products for item in t]
    #print("\nMost viewed products: ")
    #print(most_viewed_products)
    
    
    #Combining the most sold and most viewed products to form popular products
    #Using shuffling to increase randomness
    popular_products = most_sold_products + most_viewed_products
    #print("Before: ")
    #print(popular_products)
    popular_products = remove_duplicates(popular_products)
    #print("\nAfter removing duplicates: ")
    #print(popular_products)
    random.shuffle(popular_products)
    #print("\nShuffled: ")
    #print(popular_products)
    
    
    
    #FINAL STEP: Processing all the products from above steps to form the final list of the 
    #products to be recommended
    
    final_products_to_be_recommended = []
    
    #If the stock is lower than 10, the product won't be recommended
    min_stock = 10
    
    #Taking only the first 15 products from each list to limit the number of products to be recommended
    category_related_products = category_related_products[0:15]
    keyword_related_products = keyword_related_products[0:15]
    #Shuffling the list to increase randommness
    random.shuffle(keyword_related_products)
    
    #Combining both the list to perform processing
    temp_list = keyword_related_products + category_related_products
    #Removing duplicate id's
    temp_list = remove_duplicates(temp_list)
    
    
    #Retrieving the value of the average sold products from the db
    query = "select avg(sold) from shop_product;"
    cursor.execute(query)
    avg_sold = cursor.fetchall()
    avg_sold = avg_sold[0]
    avg_sold = avg_sold[0]
    avg_sold = math.ceil(avg_sold)

    #Retrieving the value of the average viewed products from the db
    query2 = "select avg(view) from shop_product;"
    cursor.execute(query2)
    avg_view = cursor.fetchall()
    avg_view = avg_view[0]
    avg_view = avg_view[0]
    avg_view = math.ceil(avg_view)
    
    iterator = 0
    iterator2 = 0
    
    for iterator2 in range(len(category_related_products)):
        prod_id = temp_list[iterator2]
        query3 = """select status from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query3)
        status = cursor.fetchall()
        
        #This is done twice to fetch the value from the tuple to a normal variable
        status = status[0]
        status = status[0]
        
        query4 = """select stock from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query4)
        stock = cursor.fetchall()
        stock = stock[0]
        stock = stock[0]
        
        if (stock > min_stock) and (status is 1):
                final_products_to_be_recommended.append(prod_id)
        
        
    
    for iterator in range(len(popular_products)):
        prod_id = popular_products[iterator]
        query1 = """select sold from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query1)
        sold = cursor.fetchall()
        sold = sold[0]
        sold = sold[0]
        
        query2 = """select view from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query2)
        view = cursor.fetchall()
        view = view[0]
        view = view[0]
        
        query3 = """select status from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query3)
        status = cursor.fetchall()
        status = status[0]
        status = status[0]
        
        query4 = """select stock from shop_product where id ="""+str(prod_id)+""";"""
        cursor.execute(query4)
        stock = cursor.fetchall()
        stock = stock[0]
        stock = stock[0]
        
        if (sold > avg_sold) and (view > avg_view):
            if (stock > min_stock) and (status is 1):
                final_products_to_be_recommended.append(prod_id)
    
    #print("\nFinal products to recommend: ")
    #print(final_products_to_recommend)
    return final_products_to_be_recommended


In [7]:
#Getting connection to DB
#Change these values according to your own database credentials
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="imghostrider1",
    database="hundredplus"
)

#printing the DB object 
print("Database object: ")
print(mydb)
print("\n")

#Setting up the cursor
cursor = mydb.cursor()


#The value in the parameters should be the ID of the current product on the product display page
#Call to the recommending function

id_given = "669"  #This should have the ID of the current product on the product display page
final_products_to_recommend = recommend_products(id_given)


print("Product displayed: ")
cursor.execute("select product_id, name from shop_product_description where product_id ="+id_given+";")
displayed_product = cursor.fetchall()
print(displayed_product)


temp2 = []
for id in final_products_to_recommend:
    query = "select product_id, name from shop_product_description where product_id ="+str(id)+";"
    cursor.execute(query)
    temp = cursor.fetchall()
    temp2 = temp2 + temp

print("\nAll the recommended products and their ID's :\n")
print(temp2)

Database object: 


Product displayed: 
[(669, 'CELLO DAZZLE Opalware Premium Queen Cups Medium 6pcs- Livid Lilac Design')]

All the recommended products and their ID's :

[(669, 'CELLO DAZZLE Opalware Premium Queen Cups Medium 6pcs- Livid Lilac Design'), (667, 'CELLO DAZZLE Opalware Premium Queen Cups Medium 6pcs- Cool Lines Design'), (9, 'Apex Unbreakable Transparent Plastic Clear Glass Set (6 Pcs)'), (239, 'Borosil Marina Glass Jug  Microwave Safe With Handle (800 ml) (Transparent)'), (240, 'Borosil Marina Glass Jug  Microwave Safe With Handle (1.3 Litres) (Transparent)'), (241, 'Borosil Marina Designed Jug Microwave Safe with Handle (800 ml) (Transparent)'), (242, 'Borosil  Marina Designed Jug Microwave Safe with Handle (1.3 Litres) (Transparent)'), (243, 'Borosil  Marina Jug Flora Design Microwave Safe with Handle (800 ml) (Transparent)'), (244, 'Borosil Marina Jug Flora Design Microwave Safe with Handle  (1.3 Litres) (Transparent)'), (525, 'Milton stainless steel thermo mug,speci